# I Fäders spår för framtids sägrar!
using Sam's API
https://github.com/basegpu/vasapi/blob/main/readme.md

input is year, sex and rank (! not bib)

In [4]:
import requests
import json
import pandas as pd
import time
import pickle
import datetime

## example: one request

In [5]:
r = requests.get('https://vasa-api.herokuapp.com/result/2022/M/2999')
print(r.status_code)
print(r.headers)
print(r.text)

200
{'Connection': 'keep-alive', 'Server': 'gunicorn/19.10.0', 'Date': 'Sat, 10 Sep 2022 13:15:05 GMT', 'Content-Type': 'application/json', 'Content-Length': '185', 'Via': '1.1 vegur'}
{"Lopper":{"Bib":"4092","Group":"H50","Name":"Ankarhamn, P\u00e4r","Nation":"SWE","Sex":"Sex.M"},"Overall":{"Place":3161,"StartGroup":"VL4","Time":"06:07:45"},"Place":2999,"Year":2022}



# get data

## function

In [7]:
def get_data_from_api(year, sex, nr_start, nr_end, pickling=False):
    print('------------------------------------------')
    print(f'INFO: started at: {datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")}')
    print(f'--- {nr_start} to {nr_end} ---')
    t_start = time.time()
    
    i = nr_start-1 # start prior, add 1 at beginning of loop, because of 'continue' in loop
    racers = []
    
    while i < nr_end:
        i = i+1
        r = requests.get(f'https://vasa-api.herokuapp.com/result/{year}/{sex}/{i}')
        if r.status_code != 200:
            print(f'WARNING: bib not found: {i}')
            print(f'WARNING: Status Code: {r.status_code}')
            continue #stop current iteration
        
        # json2dict
        data = json.loads(r.text)
        # dict to flat list
        racer = list(data['Lopper'].values())+list(data['Overall'].values())+[data['Place'],data['Year']]
        # add to results
        racers = racers + [racer]
        
        if ((i-nr_start) == (int((nr_end-nr_start)/4))): print(f'INFO: {datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")} 1/4 way there ... {i}, {int(time.time()-t_start)}s since start, ') 
        if ((i-nr_start) == (int((nr_end-nr_start)/2))): print(f'INFO: {datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")} half way there ... {i}, {int(time.time()-t_start)}s since start, ') 
        if ((i-nr_start) == (int((nr_end-nr_start)*3/4))): print(f'INFO: {datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")} 3/4 way there ... {i}, {int(time.time()-t_start)}s since start, ') 
        
        
    # define the columns names from the keys
    keys = list(data['Lopper'].keys())+list(data['Overall'].keys())+['Place_cat','Year']
    
    # create df
    df = pd.DataFrame(racers, columns=keys)
    elapsed_time = time.time()-t_start
    print(f'INFO: elapsed time {int(elapsed_time)}s (= {int(elapsed_time/60)}min)')
    print(f'INFO: records: {df.shape[0]} from possible {nr_end-nr_start+1}')
    
    # pickle for framtids sägrer
    if pickling:
        print(f'INFO: pickling ...')
        filename = f'data/results_{year}_{sex}_{nr_start}_{nr_end}.pickle'
        with open(filename, 'wb') as f:
            pickle.dump(df, f)
        print(f'INFO: ... DONE: filename {filename}')
    return(df)


## for men

In [9]:
# DO IT FOR MANY, SAFE EVERY 1000 RACERS IN A FILE IN CASE OF CONNECTION TROUBLE
# FOR MEN
# 2022: 10_060 runners
#[0,1000,2000,3000,4000,5000,6000,7000,8000,9000,10_000]
for i in [0,9000,10_000]:
    df = get_data_from_api(2022,'M',i+1,i+1000, pickling=True)


------------------------------------------
INFO: started at: 11/09/2022 07:00:47
--- 1 to 1000 ---
INFO: 11/09/2022 07:06:05 1/4 way there ... 250, 318s, 
INFO: 11/09/2022 07:11:14 half way there ... 500, 627s, 
INFO: 11/09/2022 07:16:18 3/4 way there ... 750, 931s, 
INFO: elapsed time 1236s (= 20min)
INFO: records: 1000 from possible 1000
INFO: pickling ...
INFO: ... DONE: filename data/results_2022_M_1_1000.pickle
------------------------------------------
INFO: started at: 11/09/2022 07:21:23
--- 9001 to 10000 ---
INFO: 11/09/2022 07:26:41 1/4 way there ... 9250, 317s, 
INFO: 11/09/2022 07:31:56 half way there ... 9500, 633s, 
INFO: 11/09/2022 07:37:10 3/4 way there ... 9750, 947s, 
INFO: elapsed time 1254s (= 20min)
INFO: records: 1000 from possible 1000
INFO: pickling ...
INFO: ... DONE: filename data/results_2022_M_9001_10000.pickle
------------------------------------------
INFO: started at: 11/09/2022 07:42:17
--- 10001 to 11000 ---


INFO: elapsed time 677s (= 11min)
INFO: records: 60 from possible 1000
INFO: pickling ...
INFO: ... DONE: filename data/results_2022_M_10001_11000.pickle


## for women

In [10]:
# DO IT FOR MANY, SAFE EVERY 1000 RACERS IN A FILE IN CASE OF CONNECTION TROUBLE
# SAME FOR WOMEN
# 2022: 1990 runners

for i in [0,1000]:
    df = get_data_from_api(2022,'W',i+1,i+1000, pickling=True)

------------------------------------------
INFO: started at: 11/09/2022 07:53:35
--- 1 to 1000 ---
INFO: 11/09/2022 07:58:36 1/4 way there ... 250, 301s, 
INFO: 11/09/2022 08:03:51 half way there ... 500, 616s, 
INFO: 11/09/2022 08:09:03 3/4 way there ... 750, 927s, 
INFO: elapsed time 1242s (= 20min)
INFO: records: 1000 from possible 1000
INFO: pickling ...
INFO: ... DONE: filename data/results_2022_W_1_1000.pickle
------------------------------------------
INFO: started at: 11/09/2022 08:14:17
--- 1001 to 2000 ---
INFO: 11/09/2022 08:19:21 1/4 way there ... 1250, 303s, 
INFO: 11/09/2022 08:24:26 half way there ... 1500, 609s, 
INFO: 11/09/2022 08:29:33 3/4 way there ... 1750, 915s, 
INFO: elapsed time 1223s (= 20min)
INFO: records: 990 from possible 1000
INFO: pickling ...
INFO: ... DONE: filename data/results_2022_W_1001_2000.pickle
